In [ ]:
import pickle
from pathlib import Path
from mlirmut.synthfuzz.processor import UnparserRuleNode, UnlexerRuleNode, QuantifierNode, LiteralNode, AlternationNode
from mlirmut.synthfuzz.generator import QuantifierSpec, InsertMatchPattern
from math import inf

In [ ]:
with Path("/synthfuzz/eval/mlir/mlirgen/graph.pkl").open("rb") as f:
    graph = pickle.load(f)
graph

In [ ]:
parser_rules = [rule for rule in graph.rules if isinstance(rule, UnparserRuleNode)]
print(f"# parser rules: {len(parser_rules)}")
def is_simple_quantifier(node: QuantifierNode):
    return len(node.out_neighbours) == 1 and isinstance(node.out_neighbours[0], UnparserRuleNode)
def contains_quantifier(rule):
    quantifiers = [node for node in rule.out_neighbours if isinstance(node, QuantifierNode)]
    if not quantifiers:
        return False
    if all(is_simple_quantifier(node) for node in quantifiers):
        return True
    return False
parser_rules_with_quants = [rule for rule in parser_rules if contains_quantifier(rule)]
print(f"# parser rules with simple quantifiers: {len(parser_rules_with_quants)}")

# create a mapping from the rules inside the quantifiers back to the parent rule itself
# we need this to filter candidate locations by the parent node during mutation
insert_patterns = dict()
quantified_nodes = dict()
for rule in parser_rules_with_quants:
    match_pattern = list()
    child_rules = set()
    valid_rule = True
    for child in rule.out_neighbours:
        if isinstance(child, QuantifierNode):
            # TODO handle complex quantifier patterns
            child_rule: UnparserRuleNode = child.out_neighbours[0]
            if not isinstance(child_rule, UnparserRuleNode):
                print(rule.name)
                raise ValueError(f"Quantifier pattern expected to contain a rule, but found {type(child_rule)}")
            match_pattern.append(QuantifierSpec(min=child.min, max=child.max if child.max != 'inf' else inf, rule_name=child_rule.name))
            child_rules.add(child_rule.name)
        elif isinstance(child, UnparserRuleNode):
            match_pattern.append(child.name)
        elif isinstance(child, UnlexerRuleNode):
            match_pattern.append(child.name)
        elif isinstance(child, LiteralNode):
            match_pattern.append(child.src)
        elif isinstance(child, AlternationNode):
            # TODO handle alternation nodes
            valid_rule = False
            break
        else:
            print(rule.name)
            raise ValueError(f"Unexpected node type: {type(child)}")
    if valid_rule:
        insert_patterns[rule.name] = InsertMatchPattern(match_pattern, child_rules)
len(insert_patterns), insert_patterns

In [ ]:
with open("/synthfuzz/eval/mlir/mlirgen/insert_patterns.pkl", "wb") as f:
    pickle.dump(insert_patterns, f)